<a href="https://www.kaggle.com/code/pappuyadav0324/hyperparameter-tuning-of-neural-networks?scriptVersionId=268845403" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/pima-indians-diabetes-database/diabetes.csv


In [2]:
df=pd.read_csv('/kaggle/input/pima-indians-diabetes-database/diabetes.csv')

In [3]:
df.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [4]:
df.shape

(768, 9)

In [5]:
# correlation of Outcome columns with respects to other colmns
df.corr()['Outcome']

Pregnancies                 0.221898
Glucose                     0.466581
BloodPressure               0.065068
SkinThickness               0.074752
Insulin                     0.130548
BMI                         0.292695
DiabetesPedigreeFunction    0.173844
Age                         0.238356
Outcome                     1.000000
Name: Outcome, dtype: float64

In [6]:
#check for null,duplicated and nan value
df.isnull().sum(),df.duplicated().sum(),df.isna().sum()

(Pregnancies                 0
 Glucose                     0
 BloodPressure               0
 SkinThickness               0
 Insulin                     0
 BMI                         0
 DiabetesPedigreeFunction    0
 Age                         0
 Outcome                     0
 dtype: int64,
 0,
 Pregnancies                 0
 Glucose                     0
 BloodPressure               0
 SkinThickness               0
 Insulin                     0
 BMI                         0
 DiabetesPedigreeFunction    0
 Age                         0
 Outcome                     0
 dtype: int64)

In [7]:
#separating dependent and independent value
X=df.iloc[:,:-1]
y=df.iloc[:,-1]

In [8]:
#train_test_split
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.25,random_state=42)

In [9]:
#Scaling the train and test data 
from sklearn.preprocessing import MinMaxScaler
scaler=MinMaxScaler()
X_train_scaled=scaler.fit_transform(X_train)
X_test_scaled=scaler.transform(X_test)

In [10]:
import tensorflow as tf
from tensorflow import keras
from keras import Sequential
from keras.layers import Dense

2025-10-18 03:52:07.141621: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1760759527.334872      19 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1760759527.389880      19 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [11]:
#creating the neural network model
tf.random.set_seed(43)
model=Sequential()
model.add(Dense(32,activation='relu',input_dim=(X.shape[1])))
model.add(Dense(1,activation='sigmoid'))
#compile the model
loss=tf.keras.losses.BinaryCrossentropy()
optimizer=tf.keras.optimizers.Adam(learning_rate=0.001)
metrics=['accuracy']
model.compile(loss=loss,optimizer=optimizer,metrics=metrics)
#callbacks 
early_stopping=tf.keras.callbacks.EarlyStopping(monitor='accuracy',patience=5,restore_best_weights=True)
#fit the model
model.fit(X_train_scaled,y_train,epochs=100,callbacks=[early_stopping],validation_data=(X_test_scaled,y_test))

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
I0000 00:00:1760759540.336230      19 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 13942 MB memory:  -> device: 0, name: Tesla T4, pci bus id: 0000:00:04.0, compute capability: 7.5
I0000 00:00:1760759540.337123      19 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:1 with 13942 MB memory:  -> device: 1, name: Tesla T4, pci bus id: 0000:00:05.0, compute capability: 7.5


Epoch 1/100


I0000 00:00:1760759543.228217      60 service.cc:148] XLA service 0x7a5e8c008ef0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1760759543.228628      60 service.cc:156]   StreamExecutor device (0): Tesla T4, Compute Capability 7.5
I0000 00:00:1760759543.228649      60 service.cc:156]   StreamExecutor device (1): Tesla T4, Compute Capability 7.5
I0000 00:00:1760759543.449124      60 cuda_dnn.cc:529] Loaded cuDNN version 90300


 1/18 ━━━━━━━━━━━━━━━━━━━━ 38s 2s/step - accuracy: 0.2812 - loss: 0.7840

I0000 00:00:1760759544.037112      60 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


18/18 ━━━━━━━━━━━━━━━━━━━━ 3s 29ms/step - accuracy: 0.3457 - loss: 0.7479 - val_accuracy: 0.3958 - val_loss: 0.7060
Epoch 2/100
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.4908 - loss: 0.6929 - val_accuracy: 0.6302 - val_loss: 0.6756
Epoch 3/100
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.6521 - loss: 0.6644 - val_accuracy: 0.6406 - val_loss: 0.6646
Epoch 4/100
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.6595 - loss: 0.6532 - val_accuracy: 0.6406 - val_loss: 0.6605
Epoch 5/100
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.6595 - loss: 0.6476 - val_accuracy: 0.6406 - val_loss: 0.6566
Epoch 6/100
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.6595 - loss: 0.6426 - val_accuracy: 0.6406 - val_loss: 0.6528
Epoch 7/100
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.6503 - loss: 0.6379 - val_accuracy: 0.6406 - val_loss: 0.6493
Epoch 8/100
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.6503 - loss: 0.6336 - val_accuracy: 0.6406 - val_loss: 0.

# hyperparameter tuning for optimizers

In [12]:
import keras_tuner as kt
def build_model(hp):
    model1=Sequential()
    model1.add(Dense(32,activation='relu',input_dim=(X.shape[1])))
    model1.add(Dense(1,activation='sigmoid'))
    #compile the model
    optimizer=hp.Choice('otimizers',values=['adam','sgd','rmsprop','adadelta'])
    loss=tf.keras.losses.BinaryCrossentropy()
    metrics=['accuracy']
    model1.compile(optimizer=optimizer,loss=loss,metrics=metrics)
    return model1

In [13]:
tuner1=kt.RandomSearch(build_model,objective='val_accuracy',max_trials=5)

In [14]:
tuner1.search(X_train_scaled,y_train,epochs=100,callbacks=[early_stopping],validation_data=(X_test_scaled,y_test))

Trial 4 Complete [00h 00m 03s]
val_accuracy: 0.640625

Best val_accuracy So Far: 0.703125
Total elapsed time: 00h 00m 14s


In [15]:
tuner1.get_best_hyperparameters()[0].values

{'otimizers': 'sgd'}

In [16]:
#get best model
model2=tuner1.get_best_models(num_models=1)[0]

In [17]:
model2.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 32)             │           288 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 321 (1.25 KB)

 Trainable params: 321 (1.25 KB)

 Non-trainable params: 0 (0.00 B)

In [18]:
#fit the model
model2.fit(X_train_scaled,y_train,epochs=100,callbacks=[early_stopping],validation_data=(X_test_scaled,y_test))

Epoch 1/100
18/18 ━━━━━━━━━━━━━━━━━━━━ 1s 27ms/step - accuracy: 0.6686 - loss: 0.6656 - val_accuracy: 0.6719 - val_loss: 0.6626
Epoch 2/100
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.6580 - loss: 0.6617 - val_accuracy: 0.6354 - val_loss: 0.6599
Epoch 3/100
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.6577 - loss: 0.6587 - val_accuracy: 0.6406 - val_loss: 0.6578
Epoch 4/100
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.6595 - loss: 0.6563 - val_accuracy: 0.6406 - val_loss: 0.6561
Epoch 5/100
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.6595 - loss: 0.6544 - val_accuracy: 0.6406 - val_loss: 0.6548
Epoch 6/100
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.6595 - loss: 0.6527 - val_accuracy: 0.6406 - val_loss: 0.6537


# Hyperparameters tuning for Neurons

In [19]:
def build_model1(hp):
    model3=Sequential()
    units=hp.Int('units',min_value=8,max_value=128)
    model3.add(Dense(units=units,activation='relu',input_dim=(X_train.shape[1])))
    model3.add(Dense(1,activation='sigmoid'))
    #compile the model
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.001)
    model3.compile(loss=loss,optimizer=optimizer,metrics=metrics)
    return model3

In [20]:
tuner3=kt.RandomSearch(build_model1,objective='val_accuracy',max_trials=5,directory='neuron_dir1',project_name='neuron_11')

In [21]:
tuner3.search(X_train_scaled,y_train,epochs=10,validation_data=(X_test_scaled,y_test))

Trial 5 Complete [00h 00m 04s]
val_accuracy: 0.6770833134651184

Best val_accuracy So Far: 0.6770833134651184
Total elapsed time: 00h 00m 22s


In [22]:
tuner3.get_best_hyperparameters()[0].values

{'units': 57}

In [23]:
#get the best model
model3_best=tuner3.get_best_models(num_models=1)[0]

/usr/local/lib/python3.11/dist-packages/keras/src/saving/saving_lib.py:757: UserWarning: Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 10 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


In [24]:
model3_best.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 57)             │           513 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │            58 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 571 (2.23 KB)

 Trainable params: 571 (2.23 KB)

 Non-trainable params: 0 (0.00 B)

In [25]:
model3_best.fit(X_train_scaled,y_train,epochs=100,initial_epoch=11,callbacks=[early_stopping],validation_data=(X_test_scaled,y_test))

Epoch 12/100
18/18 ━━━━━━━━━━━━━━━━━━━━ 2s 26ms/step - accuracy: 0.6744 - loss: 0.5962 - val_accuracy: 0.6562 - val_loss: 0.6053
Epoch 13/100
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.6957 - loss: 0.5900 - val_accuracy: 0.6667 - val_loss: 0.6013
Epoch 14/100
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.6998 - loss: 0.5849 - val_accuracy: 0.6615 - val_loss: 0.5975
Epoch 15/100
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.7066 - loss: 0.5801 - val_accuracy: 0.6615 - val_loss: 0.5938
Epoch 16/100
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.7111 - loss: 0.5754 - val_accuracy: 0.6667 - val_loss: 0.5903
Epoch 17/100
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.7153 - loss: 0.5707 - val_accuracy: 0.6667 - val_loss: 0.5869
Epoch 18/100
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.7174 - loss: 0.5663 - val_accuracy: 0.6667 - val_loss: 0.5837
Epoch 19/100
18/18 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.7209 - loss: 0.5619 - val_accuracy: 0

# Hyperparamater tuning for layers

In [26]:
# Ensure eager execution
tf.config.run_functions_eagerly(True)
tf.compat.v1.enable_eager_execution()

def build_model4(hp):
    model4=Sequential()
    model4.add(Dense(58,activation='relu',input_dim=(X_train.shape[1])))
    for i in range(hp.Int('layers',min_value=1,max_value=10)):
        model4.add(Dense(58,activation='relu'))
    model4.add(Dense(1,activation='sigmoid'))
    optimizer = keras.optimizers.Adam(
        learning_rate=hp.Choice('learning_rate', [1e-2, 1e-3, 1e-4])
    )
    model4.compile(loss=loss,optimizer=optimizer,metrics=metrics)
    return model4

In [27]:
tuner4=kt.RandomSearch(build_model4,objective='val_accuracy' ,overwrite=True,max_trials=5,directory='layers0',project_name='layers_project0')

In [28]:
tuner4.search(X_train_scaled,y_train,epochs=10,validation_data=(X_test_scaled,y_test))

Trial 5 Complete [00h 00m 14s]
val_accuracy: 0.734375

Best val_accuracy So Far: 0.796875
Total elapsed time: 00h 01m 41s


In [29]:
tuner4.get_best_hyperparameters()[0].values

{'layers': 7, 'learning_rate': 0.001}

In [30]:
#get the best model
model4_best=tuner4.get_best_models(num_models=1)[0]

/usr/local/lib/python3.11/dist-packages/keras/src/saving/saving_lib.py:757: UserWarning: Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 38 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


In [31]:
model4_best.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 58)             │           522 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 58)             │         3,422 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 58)             │         3,422 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 58)             │         3,422 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 58)             │         3,422 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 58)             │         3,422 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_6 (Dense)                 │ (None, 58)             │         3,422 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_7 (Dense)                 │ (None, 58)             │         3,422 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_8 (Dense)                 │ (None, 1)              │            59 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 24,535 (95.84 KB)

 Trainable params: 24,535 (95.84 KB)

 Non-trainable params: 0 (0.00 B)

In [32]:
model4_best.fit(X_train_scaled,y_train,epochs=100,initial_epoch=11,callbacks=[early_stopping],validation_data=(X_test_scaled,y_test))

Epoch 12/100
18/18 ━━━━━━━━━━━━━━━━━━━━ 2s 130ms/step - accuracy: 0.7834 - loss: 0.4302 - val_accuracy: 0.7917 - val_loss: 0.5295
Epoch 13/100
18/18 ━━━━━━━━━━━━━━━━━━━━ 2s 127ms/step - accuracy: 0.7911 - loss: 0.4088 - val_accuracy: 0.7656 - val_loss: 0.5228
Epoch 14/100
18/18 ━━━━━━━━━━━━━━━━━━━━ 2s 128ms/step - accuracy: 0.8006 - loss: 0.3962 - val_accuracy: 0.7969 - val_loss: 0.5340
Epoch 15/100
18/18 ━━━━━━━━━━━━━━━━━━━━ 2s 128ms/step - accuracy: 0.8006 - loss: 0.3923 - val_accuracy: 0.7969 - val_loss: 0.5304
Epoch 16/100
18/18 ━━━━━━━━━━━━━━━━━━━━ 2s 129ms/step - accuracy: 0.8075 - loss: 0.3902 - val_accuracy: 0.7969 - val_loss: 0.5294
Epoch 17/100
18/18 ━━━━━━━━━━━━━━━━━━━━ 2s 127ms/step - accuracy: 0.8091 - loss: 0.3836 - val_accuracy: 0.7917 - val_loss: 0.5347
Epoch 18/100
18/18 ━━━━━━━━━━━━━━━━━━━━ 2s 128ms/step - accuracy: 0.8087 - loss: 0.3802 - val_accuracy: 0.7865 - val_loss: 0.5345
Epoch 19/100
18/18 ━━━━━━━━━━━━━━━━━━━━ 2s 128ms/step - accuracy: 0.8143 - loss: 0.3755 - 

# Hyperparamater tunning for all

In [33]:
import keras_tuner as kt
from tensorflow import keras
from tensorflow.keras import layers

def build_model(hp):
    model = keras.Sequential()
    input_dim = X_train_scaled.shape[1]

    # --- Number of hidden layers ---
    num_layers = hp.Int('num_layers', min_value=1, max_value=5)

    for i in range(num_layers):
        # Dense layer with tunable neurons and activation
        model.add(layers.Dense(
            units=hp.Int(f'units_{i}', min_value=16, max_value=512, step=32),
            activation=hp.Choice(f'activation_{i}', ['relu', 'tanh', 'leaky_relu', 'elu']),
            input_dim=input_dim if i == 0 else None
        ))

        # Batch Normalization toggle
        if hp.Boolean(f'batch_norm_{i}'):
            model.add(layers.BatchNormalization())

        # Dropout layer with tunable rate
        model.add(layers.Dropout(
            rate=hp.Float(f'dropout_{i}', min_value=0.0, max_value=0.5, step=0.1)
        ))

    # --- Output layer (binary classification) ---
    model.add(layers.Dense(1, activation='sigmoid'))

    # --- Optimizer tuning ---
    optimizer_choice = hp.Choice('optimizer', ['adam', 'sgd', 'rmsprop'])
    learning_rate = hp.Choice('learning_rate', [1e-2, 1e-3, 1e-4, 5e-4])

    if optimizer_choice == 'adam':
        optimizer = keras.optimizers.Adam(learning_rate=learning_rate)
    elif optimizer_choice == 'sgd':
        optimizer = keras.optimizers.SGD(
            learning_rate=learning_rate,
            momentum=hp.Choice('momentum', [0.0, 0.5, 0.9])
        )
    else:  # rmsprop
        optimizer = keras.optimizers.RMSprop(learning_rate=learning_rate)

    # --- Compile model ---
    model.compile(
        optimizer=optimizer,
        loss='binary_crossentropy',  # change to 'mse' for regression
        metrics=['accuracy']
    )

    return model


In [34]:
tuner = kt.RandomSearch(
    build_model,
    objective='val_accuracy',
    max_trials=5,  # total number of models to try
    executions_per_trial=1,  # average of runs per trial
    directory='hyper_tuner_dir1',
    project_name='deep_tuner1'
)

tuner.search(X_train_scaled, y_train, epochs=20, validation_data=(X_test_scaled, y_test))


Trial 5 Complete [00h 00m 45s]
val_accuracy: 0.7760416865348816

Best val_accuracy So Far: 0.7760416865348816
Total elapsed time: 00h 02m 54s


In [35]:
best_hp = tuner.get_best_hyperparameters(1)[0]
print("Best hyperparameters:")
for key, value in best_hp.values.items():
    print(f"{key}: {value}")

best_model = tuner.get_best_models(num_models=1)[0]
best_model.summary()


Best hyperparameters:
num_layers: 5
units_0: 80
activation_0: elu
batch_norm_0: True
dropout_0: 0.2
optimizer: sgd
learning_rate: 0.0001
units_1: 464
activation_1: tanh
batch_norm_1: True
dropout_1: 0.30000000000000004
units_2: 112
activation_2: relu
batch_norm_2: False
dropout_2: 0.4
units_3: 496
activation_3: leaky_relu
batch_norm_3: True
dropout_3: 0.4
units_4: 400
activation_4: elu
batch_norm_4: False
dropout_4: 0.30000000000000004
momentum: 0.9


/usr/local/lib/python3.11/dist-packages/keras/src/saving/saving_lib.py:757: UserWarning: Skipping variable loading for optimizer 'SGD', because it has 2 variables whereas the saved optimizer has 20 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 80)             │           720 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 80)             │           320 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 80)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 464)            │        37,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_1           │ (None, 464)            │         1,856 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 464)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 112)            │        52,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 112)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 496)            │        56,048 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_2           │ (None, 496)            │         1,984 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_3 (Dropout)             │ (None, 496)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 400)            │       198,800 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_4 (Dropout)             │ (None, 400)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 1)              │           401 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 349,793 (1.33 MB)

 Trainable params: 347,713 (1.33 MB)

 Non-trainable params: 2,080 (8.12 KB)

In [36]:
best_model.fit(X_train_scaled,y_train,epochs=100,initial_epoch=20,callbacks=[early_stopping],validation_data=(X_test_scaled,y_test))

Epoch 21/100
18/18 ━━━━━━━━━━━━━━━━━━━━ 2s 125ms/step - accuracy: 0.7089 - loss: 0.6329 - val_accuracy: 0.7604 - val_loss: 0.5311
Epoch 22/100
18/18 ━━━━━━━━━━━━━━━━━━━━ 2s 119ms/step - accuracy: 0.6834 - loss: 0.6369 - val_accuracy: 0.7604 - val_loss: 0.5263
Epoch 23/100
18/18 ━━━━━━━━━━━━━━━━━━━━ 2s 121ms/step - accuracy: 0.6904 - loss: 0.6477 - val_accuracy: 0.7500 - val_loss: 0.5212
Epoch 24/100
18/18 ━━━━━━━━━━━━━━━━━━━━ 2s 120ms/step - accuracy: 0.6869 - loss: 0.6389 - val_accuracy: 0.7500 - val_loss: 0.5164
Epoch 25/100
18/18 ━━━━━━━━━━━━━━━━━━━━ 2s 119ms/step - accuracy: 0.7398 - loss: 0.5882 - val_accuracy: 0.7448 - val_loss: 0.5141
Epoch 26/100
18/18 ━━━━━━━━━━━━━━━━━━━━ 2s 122ms/step - accuracy: 0.7139 - loss: 0.6541 - val_accuracy: 0.7448 - val_loss: 0.5107
Epoch 27/100
18/18 ━━━━━━━━━━━━━━━━━━━━ 2s 120ms/step - accuracy: 0.6855 - loss: 0.6308 - val_accuracy: 0.7448 - val_loss: 0.5092
Epoch 28/100
18/18 ━━━━━━━━━━━━━━━━━━━━ 2s 120ms/step - accuracy: 0.7337 - loss: 0.5558 - 